In [2]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

/Users/divyam/Dev/Data_Science/DSF_Project/AcademicPaperRanking
['AcademicPaperRanking', 'mag_2007.csv', 'mag_2013.csv', '2018.json', 'mag_2012.csv', 'mag_2010.csv', '2014.json', 'mag_2011.csv', '2015.json', '.DS_Store', 'mag_2015.csv', 'mag_2014.csv', 'mag_2016.csv', 'dblp.csv', 'mag_2017.csv', 'sys_argv.py', '2012.json', '2008.json', '2009.json', 'common_progress_kernel.ipynb', 'acm.csv', '2013.json', '2010.json', 'dblp.txt', 'aminer_papers_0.zip', '2007.json', '2016.json', 'mag_2018.csv', 'acm.txt', 'acm_data_processing.py', 'mag_2008.csv', '2017.json', 'mag_2009.csv']


In [3]:
df = pd.read_csv("../mag_2018.csv")

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143303 entries, 8 to 1210998
Data columns (total 16 columns):
Authors             143303 non-null object
Conference          350 non-null object
Citation_count      143303 non-null int64
Date                143303 non-null object
Expected_count      143303 non-null int64
Domain              143048 non-null object
Id                  143303 non-null int64
Journal             142537 non-null object
References          96353 non-null object
Title               143303 non-null object
Venue               143303 non-null object
Words               143302 non-null object
Year                143303 non-null int64
Popularity          143303 non-null float64
Month               143303 non-null int64
Publication_Rank    143303 non-null float64
dtypes: float64(2), int64(5), object(9)
memory usage: 18.6+ MB


In [39]:
df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')

In [9]:
def get_block_avg_usage(df):
    for dtype in ['float','int','object']:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))
        del mean_usage_b, mean_usage_mb, selected_dtype

In [6]:
df.head(5)

,Unnamed: 0,AA,C,CC,D,ECC,F,Id,J,RId,Ti,VFN,W,Y,kesEntityId,logprob
0,0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064
1,1,"[{'S': 1, 'AuId': 2618360078, 'AfN': 'technica...",NaN,1787,2018-02-23,1787,"[{'FId': 2779853757, 'FN': 'penicillic acid'},...",2789134836,"{'JN': 'world mycotoxin journal', 'JId': 70096...","[2073569767, 1989318771, 2109782899, 214495155...",a critical review of producers of small lacton...,World Mycotoxin Journal,"['acid', 'critical', 'lactone', 'moniliformin'...",2018,1349261,-16.192
2,2,"[{'AuId': 2498139363, 'S': 1, 'AuN': 'r r m pa...",NaN,1785,2018-03-01,1785,"[{'FId': 2779550045, 'FN': 'patulin'}, {'FId':...",2786183769,"{'JN': 'int j food microbiol', 'JId': 132626406}",NaN,alternative patulin pathway unproven,International Journal of Food Microbiology,"['alternative', 'pathway', 'patulin', 'unproven']",2018,1350611,-16.193
3,3,"[{'S': 1, 'AuId': 2127898042, 'AfN': 'google',...",NaN,1957,2018-04-01,1957,"[{'FId': 65885262, 'FN': 'scale space segmenta...",2412782625,"{'JN': 'pami', 'JId': 199944782}","[1686810756, 2618530766, 2194775991, 209711776...",deeplab semantic image segmentation with deep ...,IEEE Transactions on Pattern Analysis and Mach...,"['atrous', 'connected', 'convolution', 'convol...",2018,1635532,-16.388
4,4,"[{'S': 1, 'AuId': 2761591088, 'AfN': 'leiden u...",NaN,293,2018-08-01,293,"[{'FId': 2778316820, 'FN': 'photometric system...",2798336535,"{'JN': 'astron astrophys', 'JId': 205231332}","[2011301426, 2142220522, 2061939373, 205293750...",gaia data release 2 summary of the contents an...,Astronomy and Astrophysics,"['2', 'contents', 'data', 'gaia', 'properties'...",2018,1744805,-16.455


In [7]:
# Make more sensible column names
rename_dict = {"AA": "Authors", 
               "C": "Conference", 
               "CC": "Citation_count",
               "ECC": "Expected_count",
               "D": "Date", 
               "F": "Domain", 
               "J": "Journal", 
               "RId": "References", 
               "Ti": "Title", 
               "VFN": "Venue", 
               "W": "Words", 
               "Y": "Year", 
               "logprob":"Popularity",
               "Unnamed: 0": "Row_num"}
df.rename(index=str, columns=rename_dict, inplace=True)

In [8]:
df.select_dtypes(include="number").describe()

,Row_num,Citation_count,Expected_count,Id,Year,kesEntityId,Popularity
count,1.211000e+06,1.211000e+06,1.211000e+06,1.211000e+06,1211000.0,1.211000e+06,1.211000e+06
mean,6.054995e+05,1.064620e+00,1.064997e+00,2.797143e+09,2018.0,2.260905e+08,-2.118847e+01
std,3.495857e+05,1.755177e+01,1.755261e+01,1.486480e+08,0.0,9.040128e+07,6.017534e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,4.727120e+05,2018.0,1.190005e+06,-2.200200e+01
25%,3.027498e+05,0.000000e+00,0.000000e+00,2.785024e+09,2018.0,1.142518e+08,-2.167500e+01
50%,6.054995e+05,0.000000e+00,0.000000e+00,2.799726e+09,2018.0,2.779343e+08,-2.129900e+01
75%,9.082492e+05,1.000000e+00,1.000000e+00,2.883550e+09,2018.0,2.864866e+08,-2.081000e+01
max,1.210999e+06,6.651000e+03,6.651000e+03,2.898614e+09,2018.0,2.941568e+08,-1.606400e+01


In [9]:
df.select_dtypes(exclude="number").describe()

,Authors,Conference,Date,Domain,Journal,References,Title,Venue,Words
count,1211000,50789,1211000,1197152,1019164,835626,1211000,1147111,1210997
unique,1145040,1256,363,1105872,18787,828087,1208460,42772,1207408
top,"[{'S': 1, 'AuId': 2617818522, 'AfN': 'beckman ...","{'CN': 'cleo', 'CId': 2623670637}",2018-01-01,"[{'FId': 554144382, 'FN': 'performance art'}, ...","{'JN': 'plos one', 'JId': 202381698}","[2028520323, 1657433554, 2782514705, 273981532...",open access is there a predator at the door,PLOS ONE,"['access', 'door', 'open', 'predator']"
freq,3810,1643,182147,4454,14841,1206,11,14841,11


In [10]:
df[df.duplicated(["Title"])].sort_values(by=["Title"])

,Row_num,Authors,Conference,Citation_count,Date,Expected_count,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity
507570,507570,"[{'AuId': 2784299526, 'S': 1, 'AuN': 'matthew ...",NaN,0,2018-01-18,0,"[{'FId': 2778248108, 'FN': 'anemia'}, {'FId': ...",2792264832,"{'JN': 'inflamm bowel dis', 'JId': 45123234}",NaN,1 effectiveness of oral iron supplementation i...,Inflammatory Bowel Diseases,"['1', 'anemia', 'colitis', 'effectiveness', 'f...",2018,274768078,-21.147
558615,558615,"[{'AuId': 2784842589, 'S': 1, 'AuN': 'hari k s...",NaN,0,2018-01-01,0,"[{'FId': 117717151, 'FN': 'epigenome'}, {'FId'...",2784548192,"{'JN': 'gastroenterology', 'JId': 143352558}",NaN,10 epigenome wide association study identified...,Gastroenterology,"['10', 'association', 'contributions', 'crohn'...",2018,276377826,-21.226
762005,762005,"[{'AuId': 2288827849, 'S': 1, 'AuN': 'masahiro...",NaN,0,2018-02-08,0,"[{'FId': 115171683, 'FN': 'laser beam quality'...",2787938164,"{'JN': 'japan society applied physics', 'JId':...",NaN,10 w high power and high beam quality pulsed o...,The Japan Society of Applied Physics,"['10', 'beam', 'crystal', 'double', 'emitting'...",2018,282541210,-21.508
636156,636156,"[{'AuId': 2330033061, 'S': 1, 'AuN': 'bharati ...",NaN,0,2018-01-01,0,"[{'FId': 2780101441, 'FN': 'pouch'}, {'FId': 8...",2785775086,"{'JN': 'gastroenterology', 'JId': 143352558}",NaN,11 delayed ileal pouch anal anastomosis has a ...,Gastroenterology,"['11', '30', 'adverse', 'anal', 'anastomosis',...",2018,278869938,-21.343
968830,968830,"[{'AuId': 2313863126, 'S': 1, 'AuN': 'edward l...",NaN,0,2018-01-18,0,"[{'FId': 2778260677, 'FN': 'inflammatory bowel...",2793601311,"{'JN': 'inflamm bowel dis', 'JId': 45123234}",NaN,12 high patient activation is associated with ...,Inflammatory Bowel Diseases,"['12', 'activation', 'bowel', 'clinical', 'coh...",2018,288239735,-21.749
996665,996665,"[{'AuId': 2468668054, 'S': 1, 'AuN': 'bruce e ...",NaN,0,2018-01-18,0,"[{'FId': 2778886723, 'FN': 'tofacitinib'}, {'F...",2792177297,"{'JN': 'inflamm bowel dis', 'JId': 45123234}",NaN,14 post hoc analysis of tofacitinib crohn s di...,Inflammatory Bowel Diseases,"['14', '2', 'baseline', 'biomarker', 'crohn', ...",2018,288998019,-21.783
523872,523872,"[{'AuId': 2124093779, 'S': 1, 'AuN': 'dalin li...",NaN,0,2018-01-01,0,"[{'FId': 84597430, 'FN': 'locus'}, {'FId': 230...",2783148383,"{'JN': 'gastroenterology', 'JId': 143352558}",NaN,16 genetic analysis identified novel loci asso...,Gastroenterology,"['16', 'genetic', 'ibd', 'identified', 'loci',...",2018,275294553,-21.173
725820,725820,"[{'AuId': 2787812110, 'S': 1, 'AuN': 'connie w...",NaN,0,2018-01-01,0,"[{'FId': 174749747, 'FN': 'single cell sequenc...",2786317888,"{'JN': 'gastroenterology', 'JId': 143352558}",NaN,17 single cell sequencing of creeping fat in c...,Gastroenterology,"['17', 'bacteria', 'cell', 'cellular', 'cluste...",2018,281561544,-21.463
947431,947431,"[{'AuId': 2287326182, 'S': 1, 'AuN': 'laurent ...",NaN,0,2018-01-18,0,"[{'FId': 2777061241, 'FN': 'etrolizumab'}, {'F...",2791922733,"{'JN': 'inflamm bowel dis', 'JId': 45123234}",NaN,20 etrolizumab improved endoscopic score patie...,Inflammatory Bowel Diseases,"['20', 'antagonist', 'biomarkers', 'cohort', '...",2018,287560238,-21.722
118676,118676,"[{'S': 1, 'AuId': 2138456590, 'AfN': 'universi...",NaN,2,2018-01-01,2,"[{'FId': 84393581, 'FN': 'blood pressure'}, {'...",2785069458,"{'JN': 'j am soc hypertens', 'JId': 114425954}","[2768267412, 2612462276, 2555057407, 1990090475]",2017 acc aha aapa abc acpm ags apha ash aspc n...,Journal of The American Society of Hypertension,"['2017', 'aapa', 'abc', 'acc', 'acpm', 'adults...",2018,80162441,-20.371


In [11]:
# df 
df.drop_duplicates(subset=['Title'], inplace=True)

In [17]:
df.isna().sum()

Authors                 0
Conference        1157829
Citation_count          0
Date                    0
Expected_count          0
Domain              13793
Id                      0
Journal            191087
References         373689
Title                   0
Venue               63804
Words                   3
Year                    0
Popularity              0
Month                   0
dtype: int64

In [11]:
df["Conference"].value_counts(dropna=False)

NaN                                         1157829
{'CN': 'cleo', 'CId': 2623670637}              1626
{'CN': 'chi', 'CId': 1163450153}               1467
{'CN': 'icra', 'CId': 1163902177}              1259
{'CN': 'icassp', 'CId': 1121227772}            1137
{'CN': 'icc', 'CId': 1130451194}               1026
{'CN': 'cvpr', 'CId': 1158167855}              1003
{'CN': 'aaai', 'CId': 1184914352}               872
{'CN': 'nips', 'CId': 1127325140}               843
{'CN': 'acc', 'CId': 1190039108}                792
{'CN': 'ijcai', 'CId': 1203999783}              626
{'CN': 'icml', 'CId': 1180662882}               622
{'CN': 'iscas', 'CId': 1192710900}              543
{'CN': 'icse', 'CId': 1174403976}               427
{'CN': 'ccc', 'CId': 2623792093}                414
{'CN': 'interspeech', 'CId': 1177287137}        411
{'CN': 'ofc', 'CId': 2621451516}                404
{'CN': 'naacl', 'CId': 1173951661}              394
{'CN': 'eccv', 'CId': 1124077590}               385
{'CN': 'icip

In [12]:
df.iloc[0:1,-1]

0   -16.064
Name: Popularity, dtype: float64

In [13]:
df["kesEntityId"].is_unique

True

In [14]:
df["Id"].is_unique

True

In [13]:
# expected citation count is almost same as citation count
df[df["Citation_count"] != (df["Expected_count"])]

,Row_num,Authors,Conference,Citation_count,Date,Expected_count,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity
134,134,"[{'AuId': 2778608201, 'S': 1, 'AuN': 'f arenou...",NaN,33,2018-08-01,50,"[{'FId': 202799725, 'FN': 'accuracy and precis...",2799268730,"{'JN': 'astron astrophys', 'JId': 205231332}","[2111817932, 2142220522, 2798336535, 213372565...",gaia data release 2 catalogue validation,Astronomy and Astrophysics,"['2', 'catalogue', 'data', 'gaia', 'release', ...",2018,10136560,-18.243
228,228,"[{'S': 1, 'AuId': 2070030126, 'AfN': 'columbia...",NaN,48,2018-01-25,50,"[{'FId': 2778247410, 'FN': 'solanezumab'}, {'F...",2787612390,"{'JN': 'n engl j med', 'JId': 62468778}","[1847168837, 2156220037, 2063808031, 232933607...",trial of solanezumab for mild dementia due to ...,The New England Journal of Medicine,"['alzheimer', 'dementia', 'disease', 'mild', '...",2018,12076216,-18.422
298,298,"[{'S': 1, 'AuId': 2153068280, 'AfN': 'universi...",NaN,34,2018-01-01,50,"[{'FId': 189206191, 'FN': 'genomics'}, {'FId':...",2609984900,"{'JN': 'amer j psychiat', 'JId': 116025658}","[2134783591, 2113784410, 2098597355, 203315153...",psychiatric genomics an update and an agenda,American Journal of Psychiatry,"['agenda', 'genomics', 'psychiatric', 'update']",2018,13182132,-18.511
385,385,"[{'S': 1, 'AuId': 2104561707, 'AfN': 'arthriti...",NaN,36,2018-06-01,50,"[{'FId': 137886200, 'FN': 'biopsychosocial mod...",2791379829,"{'JN': 'lancet', 'JId': 49861241}","[2296986887, 2157244868, 2160769294, 258841006...",prevention and treatment of low back pain evid...,The Lancet,"['back', 'challenges', 'directions', 'evidence...",2018,14520434,-18.610
423,423,"[{'S': 1, 'AuId': 2273142510, 'AfN': 'liverpoo...",NaN,33,2018-09-14,50,"[{'FId': 125857072, 'FN': 'star formation'}, {...",2775727542,"{'JN': 'annu rev astron astrophys', 'JId': 298...",[2773772132],multiple stellar populations in globular clusters,Annual Review of Astronomy and Astrophysics,"['clusters', 'globular', 'multiple', 'populati...",2018,14932355,-18.640
1088,1088,"[{'AuId': 1978048194, 'S': 1, 'AuN': 'jessica ...",NaN,33,2018-03-07,50,"[{'FId': 63830546, 'FN': 'scientific method'},...",2791681873,"{'JN': 'nature', 'JId': 137773608}","[2156098321, 2126930838, 2157823046, 200044254...",meta analysis and the science of research synt...,Nature,"['meta', 'research', 'science', 'synthesis']",2018,19486089,-18.916
1655,1655,"[{'AuId': 2133441925, 'S': 1, 'AuN': 'steven r...",NaN,35,2018-02-01,50,"[{'FId': 2778715236, 'FN': 'discontinuation'},...",2766957057,"{'JN': 'jbmr', 'JId': 128199769}","[2044178924, 2160005424, 2482202957, 216305816...",vertebral fractures after discontinuation of d...,Journal of Bone and Mineral Research,"['controlled', 'denosumab', 'discontinuation',...",2018,21549747,-19.020
1966,1966,"[{'S': 1, 'AuId': 2319516946, 'AfN': 'delft un...",NaN,35,2018-04-06,50,"[{'FId': 22322919, 'FN': 'genomic organization...",2788376770,"{'JN': 'science', 'JId': 3880285}","[2073504534, 1995311028, 1964441323, 212108365...",real time imaging of dna loop extrusion by con...,Science,"['condensin', 'dna', 'extrusion', 'imaging', '...",2018,22391951,-19.061
2372,2372,"[{'S': 1, 'AuId': 2147956303, 'AfN': 'universi...",NaN,48,2018-05-01,50,"[{'FId': 73949472, 'FN': 'multijunction photov...",2800899574,"{'JN': 'nature energy', 'JId': 2764528046}","[2096320304, 2071134066, 2030106935, 261520321...",high fabrication yield organic tandem photovol...,Nature Energy,"['15', 'combining', 'efficiency', 'fabrication...",2018,23762922,-19.113
2458,2458,"[{'S': 1, 'AuId': 2177864800, 'AfN': 'translat...",NaN,34,2018-02-01,50,"[{'FId': 2779829264, 'FN': 'dulaglutide'}, {'F...",2787255324,"{'JN': 'lancet diabetes endocrinol', 'JId': 25...","[2059158127, 1526927233, 2564042923, 251951039...",semaglutide versus dulaglutide once weekly in ...,The Lancet Diabetes & Endocrinology,"['2', '3b', '7', 'diabetes', 'dulaglutide', 'l...",2018,23987300,-19.124


In [14]:
# create month field
df["Month"] = pd.to_datetime(df["Date"]).dt.month

In [33]:
df['Month'].head(5)

0    1
1    2
2    3
3    4
4    8
Name: Month, dtype: int64

In [15]:
def clean_data(df):
    #removing kesEntityId as simillar fields are already present
    df.drop(columns=["Row_num","kesEntityId"],inplace=True)
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df["Month"] = df["Month"].apply(pd.to_numeric,downcast='unsigned')
    df.dropna(subset=['Venue', 'Title', 'Authors',], inplace=True)

In [37]:
clean_data(df)

KeyError: "labels ['Row_num' 'kesEntityId'] not contained in axis"

In [35]:
df_pub = pd.read_csv("Data/ProcessedScopusData.csv")

In [32]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.Venue.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.Venue.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["Venue"]],axis = 1)
    return df

def extract_field(row):
    val = row["Domain"]
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

def get_num_authors(row):
    vals = row["Authors"].split("},")
    return len(vals)

In [36]:
df = append_pub_score(df)

Unique Publications from Scopus: 1231
Publications present in Dataset: 904
Shape Before (1208460, 15)
Shape After (143303, 15)


/Users/divyam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [32]:
label_dict = {"C":0,"J":1,"CJ":2,"O":3}
def find_conference_type(row):
    if pd.isnull(row["Conference"]) and pd.isnull(row["Journal"]):
        return label_dict["O"]
    elif pd.isnull(row["Conference"]):
        return label_dict["J"]
    elif pd.isnull(row["Journal"]):
        return label_dict["C"]
    else:
        return label_dict["CJ"]

In [33]:
df["Publication Type"] = df.apply(lambda row:find_conference_type(row),axis=1)
df["Number_Of_authors"] = df.apply(lambda row:get_num_authors(row), axis = 1)

In [34]:
df.shape

(1208460, 18)

In [35]:
def tidy_split(df, column, sep='|', keep=False):
    indexes = list()
    id_values = list()
    name_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            id_values.append(presplit)
        for value in values:
            indexes.append(i)
            val = value
            index1 = val.find("AuId")
            index2 = val.find("AfN")
            index3 = val.find("'S'")
            if index3 < index1:
                val = val[index1+4:index2]
                val = val.strip("', :")
            else:
                val = val[index1+4:index3]
                val = val.strip("', :")                
            id_values.append(val)
            index1 = value.find("AuN")
            val = value[index1+3:]
            val = val.strip("': }]")
            name_values.append(val)
    new_df = df.iloc[indexes, :].copy()
    new_df[column+"_Id"] = id_values
    new_new_df = new_df.iloc[:, :].copy()
    new_new_df[column+"_Name"] = name_values
    return new_new_df

In [36]:
df_big = tidy_split(df,"Authors","},")

In [37]:
df_big.head(10)

,Authors,Conference,Citation_count,Date,Expected_count,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity,Month,Publication Type,Number_Of_authors,Authors_Id,Authors_Name
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2592382599,lubing bai
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2558648277,ze wang
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2751351928,yamin han
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2754888086,zongyan zuo
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2486643139,bin liu
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2416985679,mengna yu
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2585848087,haijuan zhang
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2609093794,jinyi lin
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2586613702,yingdong xia
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 

In [30]:
df_big.shape

NameError: name 'df_big' is not defined

In [42]:
adj_list = {}
for key,row in df_big.iterrows():
    curr_list = adj_list.get(row["Id"], [])
    curr_list.extend(list(map(int,re.findall(r'\d+', df["References"][0]))))
    curr_list = list(set(curr_list))
    adj_list[key] = curr_list
adj_list

ValueError: invalid literal for int() with base 10: '[2057327293'

In [18]:
df["References"][0]

'[2057327293, 2011472471, 2729523528, 2313405434, 2396261957, 2277225938, 2322157815, 2041121873, 2273633940, 2241954069, 2268311649, 2151915808, 2108889749, 1968551245, 2553319127, 2346653510, 2610638123, 2595338727, 2470306041, 2493461890, 2534472385]'

In [19]:
type(df["References"][0])

str

In [22]:
import re
[s for s in df["References"][0].split()]

['[2057327293,',
 '2011472471,',
 '2729523528,',
 '2313405434,',
 '2396261957,',
 '2277225938,',
 '2322157815,',
 '2041121873,',
 '2273633940,',
 '2241954069,',
 '2268311649,',
 '2151915808,',
 '2108889749,',
 '1968551245,',
 '2553319127,',
 '2346653510,',
 '2610638123,',
 '2595338727,',
 '2470306041,',
 '2493461890,',
 '2534472385]']

In [47]:
curr_list = []
curr_list.extend(map(int,re.findall(r'\d+', df["References"][0])))

In [48]:
curr_list.extend(map(int,re.findall(r'\d+', df["References"][0])))

[2027545702,
 2127685726,
 2143559032,
 2125963799,
 2078251659,
 2112876622,
 2092646281,
 2141214317,
 2046658390,
 2151554611,
 2027498662,
 2081897332,
 1980771473,
 2120733999,
 1980154502,
 2106791445,
 2047208492,
 2222306685,
 2000336618,
 1881390415,
 1881009043,
 2335913920,
 2289667014,
 2104563699,
 2113428683,
 1908153784,
 2320330796,
 2266909385,
 2038485127,
 2133275424,
 216121951,
 2096074908,
 2094778224,
 2141769755,
 2047033934,
 2128390124,
 2332403844,
 2467713285,
 2578013986,
 2138363476,
 2277396963,
 2325297898,
 2114985397,
 2072160870,
 1985982900,
 2115399503,
 2306227468,
 1979815055,
 2318156787,
 1732930179,
 2510623134,
 2419146391,
 2118542945,
 1621802947,
 1505139021,
 1972718711,
 2284028703,
 2094986548,
 2617500612,
 1874253413,
 2296452901,
 2403969263,
 2029731118,
 1999996561,
 2518041763,
 1971991764,
 2217275225,
 2472585716,
 2527946790,
 2029944598,
 2006747389,
 2180748603,
 2158806421,
 2519498039,
 2011437704,
 2744914748,
 2499660576,


In [45]:
list(map(int,re.findall(r'\d+', df["References"][0])))

[2027545702,
 2127685726,
 2143559032,
 2125963799,
 2078251659,
 2112876622,
 2092646281,
 2141214317,
 2046658390,
 2151554611,
 2027498662,
 2081897332,
 1980771473,
 2120733999,
 1980154502,
 2106791445,
 2047208492,
 2222306685,
 2000336618,
 1881390415,
 1881009043,
 2335913920,
 2289667014,
 2104563699,
 2113428683,
 1908153784,
 2320330796,
 2266909385,
 2038485127,
 2133275424,
 216121951,
 2096074908,
 2094778224,
 2141769755,
 2047033934,
 2128390124,
 2332403844,
 2467713285,
 2578013986,
 2138363476,
 2277396963,
 2325297898,
 2114985397,
 2072160870,
 1985982900,
 2115399503,
 2306227468,
 1979815055,
 2318156787,
 1732930179,
 2510623134,
 2419146391,
 2118542945,
 1621802947,
 1505139021,
 1972718711,
 2284028703,
 2094986548,
 2617500612,
 1874253413,
 2296452901,
 2403969263,
 2029731118,
 1999996561,
 2518041763,
 1971991764,
 2217275225,
 2472585716,
 2527946790,
 2029944598,
 2006747389,
 2180748603,
 2158806421,
 2519498039,
 2011437704,
 2744914748,
 2499660576,
